### Concatena as frases, identificando o genero e a duração de cada resposta

In [10]:
import pandas as pd
import numpy as np
import os
import csv

def criar_pasta_tratamento():
    """
    Cria a pasta logs_em_tratamento se ela não existir
    """
    pasta_destino = "logs_em_tratamento"
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)
    return pasta_destino

def carregar_blocos_randomizados():
    """
    Carrega o arquivo com as frases e seus respectivos blocos
    """
    blocos_path = "../dados/MQD_1465_blocos_randomizados.csv"
    return pd.read_csv(
        blocos_path,
        sep='\t',
        quoting=csv.QUOTE_ALL,
        quotechar='"',
        encoding='utf-8'
    )

def extrair_genero(df):
    """
    Extrai o gênero dos participantes do arquivo de log
    """
    genero_data = df[
        (df["Label"] == "genero") &
        (df["PennElementName"] == "selecionaGenero") &
        (df["Parameter"] == "Selected")
    ][["ParticipantMD5", "Value"]].copy()
    
    genero_data["GeneroCod"] = genero_data["Value"].str.lower().map(
        lambda g: "m" if "masculino" in g else "f"
    )
    
    return genero_data[["ParticipantMD5", "GeneroCod"]]

def processar_arquivo_log(file_path, frases_bloco, num_bloco):
    """
    Processa um arquivo de log substituindo as frases originais pelas do bloco correto
    e calcula a duração dos trials
    """
    try:
        # Lê o arquivo de log pulando as 19 linhas de cabeçalho
        df = pd.read_csv(file_path, skiprows=19, header=None)
        
        # Define as colunas apenas uma vez
        df.columns = [
            "ReceptionTime", "ParticipantMD5", "Controller", "ItemNumber", 
            "InnerElementNumber", "Label", "Group", "PennElementType", 
            "PennElementName", "Parameter", "Value", "EventTime", "Comments"
        ]
        
        # Debug: verifica se as colunas estão corretas
        #print("\nVerificando dados após renomeação:")
        #print(f"Número de colunas: {len(df.columns)}")
        #print(f"Tipos de dados:\n{df.dtypes}")
        
        # Converte EventTime para numérico
        df["EventTime"] = pd.to_numeric(df["EventTime"], errors="coerce")
        
        # Extrai informações de gênero
        generos = extrair_genero(df)
        
        # Filtra eventos de início e fim dos trials
        df_trials = df[
            (df["Label"] == "frases") & 
            (df["Value"].isin(["Start", "End"]))
        ].copy()
        
        # Ordena por participante, item e tempo
        df_trials = df_trials.sort_values(["ParticipantMD5", "ItemNumber", "EventTime"])
        
        # Debug: verifica dados dos trials
        #print(f"\nTrials encontrados: {len(df_trials)}")
        
        # Calcula duração entre Start e End para cada trial
        duracoes = []
        for (participant, item), group in df_trials.groupby(["ParticipantMD5", "ItemNumber"]):
            if len(group) == 2:  # Verifica se tem tanto Start quanto End
                start_time = group[group["Value"] == "Start"]["EventTime"].iloc[0]
                end_time = group[group["Value"] == "End"]["EventTime"].iloc[0]
                duracao = (end_time - start_time) / 1000.0  # Converte para segundos
                duracoes.append({
                    "ParticipantMD5": participant,
                    "ItemNumber": item,
                    "duracao": duracao
                })
        
        df_duracoes = pd.DataFrame(duracoes)
        
        # Debug: verifica durações calculadas
        #print(f"Durações calculadas: {len(df_duracoes)}")
        
        # Filtra apenas as linhas de classificação
        df_classificacoes = df[
            (df["Label"] == "frases") & 
            (df["Parameter"] == "Selection")
        ].copy()
        
        # Ajusta o ItemNumber baseado no número do bloco
        if num_bloco == 1:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3
        else:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3 + ((num_bloco - 1) * 150)
        
        # Merge com as frases do bloco
        df_final = df_classificacoes.merge(
            frases_bloco[["id", "frase"]],
            left_on="ItemNumber",
            right_on="id",
            how="inner"
        )
        
        # Adiciona informação de gênero
        df_final = df_final.merge(
            generos,
            on="ParticipantMD5",
            how="left"
        )
        
        # Adiciona informação de duração
        df_final = df_final.merge(
            df_duracoes,
            on=["ParticipantMD5", "ItemNumber"],
            how="left"
        )
        
        return df_final[["ParticipantMD5", "GeneroCod", "frase", "Value", "duracao"]]
        
    except Exception as e:
        print(f"Erro ao processar arquivo: {str(e)}")
        print(f"Local do erro: {e.__traceback__.tb_lineno}")
        raise
def processar_todos_arquivos():
    """
    Processa todos os arquivos de log, correlacionando com os blocos corretos
    """
    pasta_destino = criar_pasta_tratamento()
    df_blocos = carregar_blocos_randomizados()
    
    # Para cada arquivo de log (0 a 9)
    for i in range(10):
        num_bloco = i + 1
        if i == 0:
            arquivo_origem = "logs_brutos/results_prod.csv"
        else:
            arquivo_origem = f"logs_brutos/results_prod ({i}).csv"
            
        arquivo_destino = f"{pasta_destino}/bloco_{num_bloco}_concatenado.csv"
        
        if os.path.exists(arquivo_origem):
            print(f"\nProcessando bloco {num_bloco} - Arquivo: {arquivo_origem}")
            
            frases_bloco = df_blocos[df_blocos['bloco'] == num_bloco].copy()
            print(f"Frases encontradas no bloco {num_bloco}: {len(frases_bloco)}")
            
            try:
                df_processado = processar_arquivo_log(arquivo_origem, frases_bloco, num_bloco)
                
                if len(df_processado) > 0:
                    df_processado.to_csv(
                        arquivo_destino,
                        sep='\t',
                        index=False,
                        quoting=csv.QUOTE_ALL,
                        quotechar='"',
                        encoding='utf-8'
                    )
                    print(f"Arquivo salvo: {arquivo_destino}")
                    print(f"Total de classificações: {len(df_processado)}")
                else:
                    print(f"AVISO: Nenhuma classificação encontrada para o bloco {num_bloco}")
            except Exception as e:
                print(f"Erro ao processar bloco {num_bloco}: {str(e)}")
        else:
            print(f"Arquivo não encontrado: {arquivo_origem}")

# Executa o processamento
processar_todos_arquivos()


Processando bloco 1 - Arquivo: logs_brutos/results_prod.csv
Frases encontradas no bloco 1: 150
Arquivo salvo: logs_em_tratamento/bloco_1_concatenado.csv
Total de classificações: 1350

Processando bloco 2 - Arquivo: logs_brutos/results_prod (1).csv
Frases encontradas no bloco 2: 150
Arquivo salvo: logs_em_tratamento/bloco_2_concatenado.csv
Total de classificações: 1200

Processando bloco 3 - Arquivo: logs_brutos/results_prod (2).csv
Frases encontradas no bloco 3: 150
Arquivo salvo: logs_em_tratamento/bloco_3_concatenado.csv
Total de classificações: 1800

Processando bloco 4 - Arquivo: logs_brutos/results_prod (3).csv
Frases encontradas no bloco 4: 150
Arquivo salvo: logs_em_tratamento/bloco_4_concatenado.csv
Total de classificações: 1200

Processando bloco 5 - Arquivo: logs_brutos/results_prod (4).csv
Frases encontradas no bloco 5: 150
Arquivo salvo: logs_em_tratamento/bloco_5_concatenado.csv
Total de classificações: 1350

Processando bloco 6 - Arquivo: logs_brutos/results_prod (5).csv

In [1]:
import pandas as pd
import numpy as np
import os
import csv

def criar_pasta_tratamento():
    """
    Cria a pasta logs_em_tratamento se ela não existir
    """
    pasta_destino = "logs_em_tratamento"
    if not os.path.exists(pasta_destino):
        os.makedirs(pasta_destino)
    return pasta_destino

def carregar_blocos_randomizados():
    """
    Carrega o arquivo com as frases e seus respectivos blocos
    """
    blocos_path = "../dados/MQD_1465_blocos_randomizados.csv"
    return pd.read_csv(
        blocos_path,
        sep='\t',
        quoting=csv.QUOTE_ALL,
        quotechar='"',
        encoding='utf-8'
    )

def extrair_genero(df):
    """
    Extrai o gênero dos participantes do arquivo de log
    """
    genero_data = df[
        (df["Label"] == "genero") &
        (df["PennElementName"] == "selecionaGenero") &
        (df["Parameter"] == "Selected")
    ][["ParticipantMD5", "Value"]].copy()
    
    genero_data["GeneroCod"] = genero_data["Value"].str.lower().map(
        lambda g: "m" if "masculino" in g else "f"
    )
    
    return genero_data[["ParticipantMD5", "GeneroCod"]]

def processar_arquivo_log(file_path, frases_bloco, num_bloco):
    """
    Processa um arquivo de log substituindo as frases originais pelas do bloco correto
    e calcula a duração dos trials
    """
    try:
        # Lê o arquivo de log pulando as 19 linhas de cabeçalho
        df = pd.read_csv(file_path, skiprows=19, header=None)
        
        # Define as colunas apenas uma vez
        df.columns = [
            "ReceptionTime", "ParticipantMD5", "Controller", "ItemNumber", 
            "InnerElementNumber", "Label", "Group", "PennElementType", 
            "PennElementName", "Parameter", "Value", "EventTime", "Comments"
        ]
        
        # Converte EventTime para numérico
        df["EventTime"] = pd.to_numeric(df["EventTime"], errors="coerce")
        
        # Extrai informações de gênero
        generos = extrair_genero(df)
        
        # Filtra eventos de início e fim dos trials
        df_trials = df[
            (df["Label"] == "frases") & 
            (df["Value"].isin(["Start", "End"]))
        ].copy()
        
        # Ajusta o ItemNumber nos trials de acordo com o bloco
        if num_bloco == 1:
            df_trials["ItemNumber"] = df_trials["ItemNumber"] - 3
        else:
            df_trials["ItemNumber"] = df_trials["ItemNumber"] - 3 + ((num_bloco - 1) * 150)
        
        # Ordena por participante, item e tempo
        df_trials = df_trials.sort_values(["ParticipantMD5", "ItemNumber", "EventTime"])
        
        # Calcula duração entre Start e End para cada trial
        duracoes = []
        for (participant, item), group in df_trials.groupby(["ParticipantMD5", "ItemNumber"]):
            if len(group) == 2:  # Verifica se tem tanto Start quanto End
                start_time = group[group["Value"] == "Start"]["EventTime"].iloc[0]
                end_time = group[group["Value"] == "End"]["EventTime"].iloc[0]
                duracao = (end_time - start_time) / 1000.0  # Converte para segundos
                duracoes.append({
                    "ParticipantMD5": participant,
                    "ItemNumber": item,
                    "duracao": duracao
                })
        
        df_duracoes = pd.DataFrame(duracoes)
        
        # Filtra apenas as linhas de classificação
        df_classificacoes = df[
            (df["Label"] == "frases") & 
            (df["Parameter"] == "Selection")
        ].copy()
        
        # Ajusta o ItemNumber baseado no número do bloco
        if num_bloco == 1:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3
        else:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3 + ((num_bloco - 1) * 150)
        
        # Merge com as frases do bloco
        df_final = df_classificacoes.merge(
            frases_bloco[["id", "frase"]],
            left_on="ItemNumber",
            right_on="id",
            how="inner"
        )
        
        # Adiciona informação de gênero
        df_final = df_final.merge(
            generos,
            on="ParticipantMD5",
            how="left"
        )
        
        # Adiciona informação de duração
        df_final = df_final.merge(
            df_duracoes,
            on=["ParticipantMD5", "ItemNumber"],
            how="left"
        )
        
        return df_final[["ParticipantMD5", "GeneroCod", "frase", "Value", "duracao"]]
        
    except Exception as e:
        print(f"Erro ao processar arquivo: {str(e)}")
        print(f"Local do erro: {e.__traceback__.tb_lineno}")
        raise

def processar_todos_arquivos():
    """
    Processa todos os arquivos de log, correlacionando com os blocos corretos
    """
    pasta_destino = criar_pasta_tratamento()
    df_blocos = carregar_blocos_randomizados()
    
    # Para cada arquivo de log (0 a 9)
    for i in range(10):
        num_bloco = i + 1
        if i == 0:
            arquivo_origem = "logs_brutos/results_prod.csv"
        else:
            arquivo_origem = f"logs_brutos/results_prod ({i}).csv"
            
        arquivo_destino = f"{pasta_destino}/bloco_{num_bloco}_concatenado.csv"
        
        if os.path.exists(arquivo_origem):
            print(f"\nProcessando bloco {num_bloco} - Arquivo: {arquivo_origem}")
            
            frases_bloco = df_blocos[df_blocos['bloco'] == num_bloco].copy()
            print(f"Frases encontradas no bloco {num_bloco}: {len(frases_bloco)}")
            
            try:
                df_processado = processar_arquivo_log(arquivo_origem, frases_bloco, num_bloco)
                
                if len(df_processado) > 0:
                    df_processado.to_csv(
                        arquivo_destino,
                        sep='\t',
                        index=False,
                        quoting=csv.QUOTE_ALL,
                        quotechar='"',
                        encoding='utf-8'
                    )
                    print(f"Arquivo salvo: {arquivo_destino}")
                    print(f"Total de classificações: {len(df_processado)}")
                else:
                    print(f"AVISO: Nenhuma classificação encontrada para o bloco {num_bloco}")
            except Exception as e:
                print(f"Erro ao processar bloco {num_bloco}: {str(e)}")
        else:
            print(f"Arquivo não encontrado: {arquivo_origem}")

# Executa o processamento
processar_todos_arquivos()


Processando bloco 1 - Arquivo: logs_brutos/results_prod.csv
Frases encontradas no bloco 1: 150
Arquivo salvo: logs_em_tratamento/bloco_1_concatenado.csv
Total de classificações: 1350

Processando bloco 2 - Arquivo: logs_brutos/results_prod (1).csv
Frases encontradas no bloco 2: 150
Arquivo salvo: logs_em_tratamento/bloco_2_concatenado.csv
Total de classificações: 1200

Processando bloco 3 - Arquivo: logs_brutos/results_prod (2).csv
Frases encontradas no bloco 3: 150
Arquivo salvo: logs_em_tratamento/bloco_3_concatenado.csv
Total de classificações: 1800

Processando bloco 4 - Arquivo: logs_brutos/results_prod (3).csv
Frases encontradas no bloco 4: 150
Arquivo salvo: logs_em_tratamento/bloco_4_concatenado.csv
Total de classificações: 1200

Processando bloco 5 - Arquivo: logs_brutos/results_prod (4).csv
Frases encontradas no bloco 5: 150
Arquivo salvo: logs_em_tratamento/bloco_5_concatenado.csv
Total de classificações: 1350

Processando bloco 6 - Arquivo: logs_brutos/results_prod (5).csv